<a href="https://colab.research.google.com/github/luckyswaminathan/MusicVibe/blob/main/tweetemotionanalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
#@title Import relevant modules
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import nltk as nl
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
nl.download('stopwords')
nl.download('punkt')
from nltk.corpus import stopwords
nl.download('wordnet')
from wordcloud import WordCloud
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,classification_report
import seaborn as sns
from gensim.models import Word2Vec

#from scikitplot.metrics import plot_confusion_matrix


# #The following lines adjust the granularity of reporting. 
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# The following line improves formatting when ouputting NumPy arrays.
np.set_printoptions(linewidth = 200)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [41]:

#imports csv
url = 'https://raw.githubusercontent.com/luckyswaminathan/MusicVibe/main/tweet_emotions.csv'

msc_df = pd.read_csv(url)
msc_df.drop_duplicates(keep='first')
msc_df.drop(columns="tweet_id", inplace = True)
msc_df = msc_df.reindex(np.random.permutation(msc_df.index))

 #defining and removing stop words
stopWords = set(stopwords.words('english'))
stopWords.add(".")
stopWords.add(":")
stopWords.add("they're")



## msc_df['content'] = (msc_df['content']).apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopWords]))
 #tokenizing each sentence in msc_df 

tknzr = TweetTokenizer()
msc_df['content'] = msc_df['content'].apply(lambda x: re.sub('[^a-zA-Z]',' ', str(x)))
msc_df['content'] = msc_df['content'].apply(lambda x: tknzr.tokenize(x.lower()))
##msc_df['content'] = (msc_df['content']).apply(word_tokenize)

lm = WordNetLemmatizer()
msc_df['content'] = msc_df['content'].apply(lambda x: [lm.lemmatize(word) for word in x if word not in (stopWords) and word.find("quot") == -1]) 







# # splitting the input/output parts



def custencoder(df):
  df.replace(to_replace="anger", value=-1, inplace=True)
  df.replace(to_replace="boredom", value=0, inplace=True)
  df.replace(to_replace="empty", value=0, inplace=True)
  df.replace(to_replace="neutral", value=0, inplace=True)
  df.replace(to_replace="enthusiasm", value=1, inplace=True)
  df.replace(to_replace="fun", value=1, inplace=True)
  df.replace(to_replace="happiness", value=1, inplace=True)
  df.replace(to_replace="hate", value=-1, inplace=True)
  df.replace(to_replace="love", value=1, inplace=True)
  df.replace(to_replace="relief", value=1, inplace=True)
  df.replace(to_replace="sadness", value=-1, inplace=True)
  df.replace(to_replace="surprise", value=1, inplace=True)
  df.replace(to_replace="worry", value=-1, inplace=True)

custencoder(msc_df['sentiment'])


rows = msc_df.shape[0]
maxTrain =int(rows*0.7)
maxVal = int(rows*0.9)


words_train = msc_df[0:maxTrain]['content']
words_val = msc_df[maxTrain+1:maxVal]['content']
words_test = msc_df[maxVal+1:-1]['content']




feelings_train = msc_df[0:maxTrain]['sentiment']
feelings_val = msc_df[maxTrain+1:maxVal]
feelings_test = msc_df[maxVal+1:-1]['sentiment']



# word_cloud = ""
# for row in words_train:
#   word_cloud+=" ".join(row)
# wordcloud = WordCloud(width = 1000, height = 500,background_color ='white',min_font_size = 10).generate(word_cloud)
# plt.imshow(wordcloud)



# sns.countplot(x=msc_df['sentiment'])



In [54]:
#using word2vec to vectorize words
## using CBOW to model this data set- since the data set is 40000 rows seems to be a better fit 
## + tweets have a bunch of weird spellings so they are omitted


size = 1000
window = 5
min_count = 10
workers = 3
sg = 0 


stemmed_tokens = pd.Series(msc_df['content']).values
# Train the Word2Vec Model
w2v_model = Word2Vec(stemmed_tokens, min_count = min_count, workers = workers, window = window, sg = sg)
w2v_model.save("word2vec.model")


print(w2v_model)




  


Word2Vec<vocab=3211, vector_size=100, alpha=0.025>


In [55]:
## working with the word2vec model


cb_w2v_model = Word2Vec.load("word2vec.model")



Word2Vec<vocab=3211, vector_size=100, alpha=0.025>
